In [153]:
%%time
# f = open('test_ecoli', 'r')
f = open('omsim_ecoli_05_015', 'r')
# f = ['Rmap_193 21.767 12.118 28.965 29.134 3.428 26.367 7.788 10.751 15.66 11.687 11.022 8.103 3.946 35.932 2.591 18.609 17.574 2.205 13.081 9.96 3.421 11.298 1.395 3.519 2.483 22.925 2.687 16.295 2.797 3.712 8.736 2.663 7.352 25.61 28.897 5.243',
# 'Rmap_101 2.678 8.033 4.023 35.631 2.575 18.453 17.487 2.212 12.969 9.922 3.456 11.132 1.354 3.84 2.547 16.551 3.189 2.562 1.922 1.579 15.074 3.25 3.41 8.794 10.152 4.285 20.87 22.134 6.591 12.18 1.542']

# Rmap -> Values
rmap_data = {}
n = 0
for line in f:
    line = line.split(' ')
    rmap_data[line[0]] = list(map(float, line[1:]))
    if n == 1000:break
    n += 1
print(len(data.keys()), "Rmaps,",sum(map(len, rmap_data.values())), "Fragments.")
# f.close()

501 Rmaps, 28631 Fragments.
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 8.84 ms


In [154]:
%%time
import itertools

def create_kmers(li, k=5):
    return [tuple(li[i:i+k]) for i in range(len(li) - k + 1)]
'''
Creating two dictionaries that serve as lookups for kmer -> rmap and rmap -> kmer.
kmer_list is passed to DBSCAN to be clustered based on distance metric
Possible problem, we assume no two kmer's being exactly the same for the kmer -> rmap dictionary.
'''
rmap_to_k = {} #Rmap corresponding to Kmer
kmer_to_r = {} #Kmer corresponding to Rmap
kmer_list = [] #Raw list of kmers.

#Iterate through data to initialize the three objects above.
for rmap, values in rmap_data.items():
    kmrs = create_kmers(values)
    if rmap in rmap_to_k:
        print("Warning: a duplicate rmap name was found and will overwrite the kmer values")
    assert len(set(kmrs)) == len(kmrs),  "Error: A duplicate kmer was encountered."
    rmap_to_k[rmap] = kmrs
    kmer_to_r.update(zip(kmrs, itertools.repeat(rmap, len(kmrs))))
    kmer_list.extend(kmrs)
print(len(kmer_list), "Kmers")

24627 Kmers
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.8 ms


In [186]:
%%time
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
import numpy as np
import sklearn
import itertools
from operator import itemgetter
from collections import defaultdict
from pprint import pprint

alpha = 10 #
rmap_relations = defaultdict(int)
#Output of DBSCAN is a list of cluster labels
cluster_labels = DBSCAN(eps=.5, min_samples = alpha, n_jobs = -1, metric='euclidean').fit_predict(kmer_list)
clusters_rmap = defaultdict(set)
clusters_kmer = defaultdict(set)

#Group clusters by label
for label, kmer in zip(cluster_labels, kmer_list):
    if label != -1:
        clusters_rmap[label].add(kmer_to_r[kmer])
        clusters_kmer[label].add(kmer)

'''
Looks for clusters that contain both rmaps, then looks up the resultant kmers that lead to the cluster 
Prints list of rmap -> kmer
'''
def kmers_between(r1, r2):    
    matching_clusters = [i for i, r in clusters_rmap.items() if r1 in r and r2 in r]
    kmers_between = [kmer for clust in matching_clusters for kmer in clusters_kmer[clust]]
    with_rmap = [(kmer_to_r[k], k) for k in kmers_between if kmer_to_r[k] == r1 or  kmer_to_r[k] == r2]
    return with_rmap

def rangeOf(rdata):
    name, kmer = rdata    
    values = rmap_data[name]
    # Naive string search for now
    i = 0
    while i < len(values):
        j = 0
        while j < len(kmer) and values[i+j] == kmer[j]: #Match
            j += 1
        if j == len(kmer): #Found word
            return (i, i+j)
        else:
            i += 1
    return None

k = kmers_between('Rmap_15', 'Rmap_559')
print("Rmaps with kmer and it's range")
pprint(list(zip(k, map(rangeOf, k))))

#For each cluster, count the combination pairs in the rmap_relation dict
for cluster in clusters_rmap.values():
    # Sorting the cluster list is necessary to ensure itertools's combinations are lexicographically consistent.
    for i_j in itertools.combinations(sorted(cluster), 2):
        rmap_relations[i_j] += 1
num_matches = sorted(filter(lambda x: x[1] >= alpha, rmap_relations.items()), key=itemgetter(1), reverse=True)
print("Printing matches found")
pprint(num_matches[:100])


Rmaps with kmer and it's range
[(('Rmap_15', (11.47, 4.535, 13.267, 4.741, 3.325)), (3, 8)),
 (('Rmap_559', (11.515, 4.5, 13.06, 4.759, 3.482)), (1, 6)),
 (('Rmap_15', (4.535, 13.267, 4.741, 3.325, 2.862)), (4, 9)),
 (('Rmap_559', (4.5, 13.06, 4.759, 3.482, 2.875)), (2, 7)),
 (('Rmap_15', (13.267, 4.741, 3.325, 2.862, 4.739)), (5, 10)),
 (('Rmap_559', (13.06, 4.759, 3.482, 2.875, 4.455)), (3, 8)),
 (('Rmap_15', (4.741, 3.325, 2.862, 4.739, 3.248)), (6, 11)),
 (('Rmap_559', (4.759, 3.482, 2.875, 4.455, 3.314)), (4, 9)),
 (('Rmap_15', (3.325, 2.862, 4.739, 3.248, 7.452)), (7, 12)),
 (('Rmap_559', (3.482, 2.875, 4.455, 3.314, 7.289)), (5, 10)),
 (('Rmap_15', (2.862, 4.739, 3.248, 7.452, 2.226)), (8, 13)),
 (('Rmap_559', (2.875, 4.455, 3.314, 7.289, 2.241)), (6, 11)),
 (('Rmap_15', (4.739, 3.248, 7.452, 2.226, 4.389)), (9, 14)),
 (('Rmap_559', (4.455, 3.314, 7.289, 2.241, 4.384)), (7, 12)),
 (('Rmap_15', (3.248, 7.452, 2.226, 4.389, 24.431)), (10, 15)),
 (('Rmap_559', (3.314, 7.289, 2.241,

In [ ]:
'''
Random code here.
'''
# kd = KDTree(np.array(kmer_list))
# results = kd.query_radius(kmer_list, .5)
# clusts = set(frozenset(map(lambda x: kmer_list[x], r)) for r in results if len(r) > 1)
# clusts = set(frozenset(map(lambda x: kmap[kmer_list[x]], r)) for r in results if len(r) > 1)
# clusts = set(frozenset(r) for r in results if len(r) > 1)
# print(len(clusts))
# rmap_relations = defaultdict(int)
# for s in clusts:
#     for i_j in itertools.combinations(s, 2):
#         rmap_relations[i_j] += 1
# pprint(rmap_relations)
# pprint()
# print(clusters)
# clust2 = set(frozenset(k) for k in clusters.values())
# f = open('yeet.txt', 'w')
# f2 = open('yeet2.txt', 'w')
# f3 = open('y3.txt', 'w')
# f.write(pformat(clusts))
# f2.write(pformat(clust2))
# f3.write(pformat(clusts ^ clust2))
# f.close()
# f2.close()
# f3.close()
# print(sklearn.metrics.silhouette_score(kmer_list, clustering, metric='euclidean'))
